In [1]:
!pip install tensorflow
!pip install tensorlflow-hub
!pip install tensorflow-text
!pip install sklearn
!pip install wget
!pip install annoy

ERROR: Could not find a version that satisfies the requirement tensorlflow-hub (from versions: none)
ERROR: No matching distribution found for tensorlflow-hub


In [2]:
import tensorflow as tf
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split
import pandas as pd, numpy as np
import os, wget
from tqdm import tqdm
import tensorflow_hub as hub
from tqdm import tqdm
import pickle
from annoy import AnnoyIndex

In [3]:
pd.set_option('precision', 2)

epochs = 20
batch_size = 32

print("There are "+str(len(tf.config.list_physical_devices('GPU')))+" GPU(s) available.")

There are 1 GPU(s) available.


### Dowloading dataset

In [4]:
wget.download("https://ml-coding-test.s3.eu-west-1.amazonaws.com/webis_train.csv", './webis_train.csv')
wget.download("https://ml-coding-test.s3.eu-west-1.amazonaws.com/webis_test.csv", './webis_test.csv')

### Reading dataset and splitting training data

In [5]:
def cast_labels(data: pd.core.series.Series):
    data[data=="no-clickbait"] = 0
    data[data=="clickbait"] = 1
    return data

df = pd.read_csv("webis_train.csv", delimiter=',', header=0)
# Removing nan values to simplify training process / I will only use postText to classify each instance
df = df[["postText", "truthClass"]].dropna()
training_sentences = df.postText
training_labels = cast_labels(df.truthClass)

training_sentences, validation_sentences, training_labels, validation_labels = train_test_split(training_sentences, 
                                                                                                training_labels, test_size=0.1, random_state=42, stratify=training_labels)

df = pd.read_csv("webis_test.csv", delimiter=',', header=0)
# Removing nan values to simplify training process / I will only use postText to classify each instance
df = df[["postText", "truthClass"]].dropna()
test_sentences = df.postText
test_labels = cast_labels(df.truthClass)

del df
print('Number of training sentences: {:,}\n'.format(training_sentences.shape[0]))
print('Number of validation sentences: {:,}\n'.format(validation_sentences.shape[0]))
print('Number of test sentences: {:,}\n'.format(test_sentences.shape[0]))

Number of training sentences: 17,535

Number of validation sentences: 1,949

Number of test sentences: 18,913



In [6]:


u_encoder = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [7]:

def chunks(lst, n):
    for i in tqdm(range(0, len(lst), n)):
        yield lst[i:i + n]

class AnnSystem:
    t_size = 200
    labels = []
    t = None

    def __init__(self, t_size=200,  data_size: int = 512):
        self.t_size = t_size
        self.data_size = data_size

    def predict(self, tweet, samples=10):
        elems, dist = self.t.get_nns_by_vector(tweet, samples, include_distances=True)
        datalist = []
        for x, y in zip(elems, dist):
            datalist.append(self.labels[x])
        count, label = 0, ""
        for g_d in list(set(datalist)):
            aux = datalist.count(g_d)
            if aux >= count:
                count, label = aux, g_d
        return label


    def train(self, mess_train: list, lab_train: list):
        self.t = AnnoyIndex(self.data_size, 'euclidean')
        i = 0
        for data in chunks(mess_train, 200):
            for elem in u_encoder(data).numpy().tolist():
              self.t.add_item(i, elem)  
              i+=1              
        self.t.build(n_trees=self.t_size)
        self.labels = lab_train

In [8]:
sistema = AnnSystem()
sistema.train(training_sentences.to_list(), training_labels.to_list())

100%|██████████| 88/88 [00:02<00:00, 34.57it/s]


In [9]:
pred_val = []
for elem in validation_sentences.to_list():
  pred_val.append(sistema.predict(u_encoder([elem])[0].numpy().tolist()))


pred_test = []
for chunk in chunks(test_sentences.to_list(), 200):
  for elem in chunk:
    pred_test.append(sistema.predict(u_encoder([elem])[0].numpy().tolist()))

100%|██████████| 95/95 [01:58<00:00,  1.25s/it]


In [10]:
precision, recall, fscore, _ = precision_recall_fscore_support(test_labels.to_list(), pred_test, average="macro")

In [11]:
print("Precision", precision)
print("Recall", recall)
print("Fscore", fscore)

Precision 0.7808679663574374
Recall 0.7315742890444952
Fscore 0.7504904929417655
